In [545]:
%load_ext autoreload
%autoreload 2
import os
import torch
import argparse
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from models.shiftnet_model import ShiftNetModel
from options.train_options import TrainOptions 
import util
from skimage.transform import resize
import numpy as np
import models
from models.networks import UnetGenerator, InceptionDown, PartialConv, InceptionUp
from models.modules.denset_net import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [546]:
dense121 = densenet121()

../models/modules/denset_net.py:218: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight.data.fill_(1)


In [575]:
x_np = -10e15*np.random.normal(0, 1, (1, 3, 256, 256))

In [576]:
x_tr = torch.FloatTensor(x_np)

In [577]:
output = dense121(x_tr)

In [584]:
for param in dense121.modules():
    print(param)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU()
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU()
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU()
        (conv1): Conv2d(9

In [543]:
def countWeights(model):
    zeros = 0
    for param in model.parameters():
        if param is not None:
            zeros += np.product(param.shape)
    return zeros

In [544]:
countWeights(dense121)

7978856

In [516]:
inception = InceptionUp(128, 32)

128 32 16


In [517]:
x_np = np.random.normal(0, 1, (1, 128, 16, 16))

In [518]:
x_tr = torch.FloatTensor(x_np)

In [519]:
output = inception(x_tr)
output.shape


torch.Size([1, 32, 32, 32])

In [520]:
inception = InceptionDown(64, 128)

64 128 16


In [521]:
x_np = np.random.normal(0, 1, (1, 64, 128, 128))

In [522]:
x_tr = torch.FloatTensor(x_np)

In [523]:
output = inception(x_tr)

torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 64, 64])
torch.Size([1, 32, 64, 64])


In [524]:
output.shape

torch.Size([1, 128, 64, 64])